# <font color=#a0a0ff>Shapefile Subset with CLIMCAPS L2 Data Using L2SS Running Locally

#### <font color=#a0a0ff>The following commands will get a copy of the L2SS code andinstall all the required Python packages. <br>It is only necessary to do these steps once. 

In [ ]:
#!git clone https://github.com/podaac/l2ss-py
#!cd l2ss-py
#!pip install packaging>=21.3
#!pip install poetry
#!poetry install 

In [1]:
from podaac.subsetter.subset import subset
import os
from shutil import copy
import netCDF4 as nc4
import numpy as np
import shapefile as shp
import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline

In [2]:
def DrawColorbar(fig,ax,p,units):
    buf = 0.01 # space between right edge of plot and left edge of colorbar
    wid = 0.01 # width of colorbar
    pos = ax.get_position()
    cbx = pos.x1 + buf
    cby = pos.bounds[1]
    cbh = pos.bounds[3]
    cax = fig.add_axes([cbx, cby, wid, cbh])
    cb  = plt.colorbar(p, cax=cax)
    b = plt.title(units)

def plotThis(fname,label):
    f = nc4.Dataset(fname,'r')
    var  = f['surf_air_temp']
    qflg = f['surf_air_temp_qc']   
    lats = f['lat']
    lons = f['lon']

    # Quality flag meanings:
    # 0=best  1=good  2=do_not_use
    varQ = var[:]
    qf = qflg[:]
    np.place(varQ, qf>1, np.nan)
  
    myExtent = [-74,-46,-15,13]   # this includes area of Brazil shapefile
    shapes = list(shpreader.Reader(fshp).geometries())
    # Draw a scatter plot
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
    ax.set_extent(myExtent)
    p = ax.scatter(lons[:], lats[:], c=varQ, cmap=plt.cm.Spectral_r, 
                   marker='.', s=150, transform=ccrs.PlateCarree())
    title = 'Surface Air Temperature (Quality Flag Applied) -- '+label
    b = plt.title(title)

    # Add coastlines and gridlines
    ax.coastlines(resolution="50m",linewidth=0.8)
    ax.add_geometries(shapes, ccrs.PlateCarree(),edgecolor='blue',facecolor='none',alpha=0.7)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=0.5, color='#777777', alpha=0.8, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlines = True
    DrawColorbar(fig,ax,p,'Kelvin')
    f.close()

### <font color=#a0a0ff>Use the result from a previous variable subset as the input file

In [3]:
path='/home/jovyan/sub_global/'
f='SNDR.SNPP.CRIMSS.20151102T1718.m06.g174.L2_CLIMCAPS_RET.std.v02_28.G.200331153124_subsetted.nc4'
fp=path+f
fin='shp_input.nc'
fout='shp_output.nc'
copy(fp,fin)
if os.path.isfile(fout): 
    os.remove(fout)

#### <font color=#a0a0ff>Source of Brazil shapefile: 
https://data.humdata.org/dataset/f5f0648e-f085-4c85-8242-26bf6c942f40/resource/2f26be26-a081-4557-8572-58545cd70e9f/download/bra_adm_ibge_2020_shp.zip

In [4]:
fshp = '/home/jovyan/Shapefiles/bra_admbnda_adm0_ibge_2020.shp'

### <font color=#a0a0ff>Run the L2 subsetter by passing relevant arguments to the <code>subset()</code> module

In [6]:
subset(fin,
       None,        # this is where the Bounding Box info would be if this were a different demo
       fout,
       variables=['surf_air_temp','surf_air_temp_qc'],
       shapefile=fshp) 

array([[-68.1, -46.8],
       [-11.3,   4.9]])

In [7]:
!h5ls -r shp_test_output.nc

/                        Group
/atrack                  Dataset {33}
/lat                     Dataset {33, 30}
/lon                     Dataset {33, 30}
/obs_time_tai93          Dataset {33, 30}
/surf_air_temp           Dataset {33, 30}
/surf_air_temp_qc        Dataset {33, 30}
/xtrack                  Dataset {30}


In [ ]:
plotThis('shp_test_output.nc','Brazil Shapefile Subset')